<a href="https://colab.research.google.com/github/WinNatch/UoB-Data-Analytics-in-Business/blob/main/Version_3_Data_Engineer_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [ ]:
df = pd.read_csv("./WA_Fn-UseC_-Telco-Customer-Churn.csv")
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [ ]:
df.info()
df['Contract'].unique()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


array(['Month-to-month', 'One year', 'Two year'], dtype=object)

In [ ]:
# New Feature 1
df['renewed_status'] = df['tenure'] - df['contract_month']

# New Feature 2
# Calculate the remaining time for one-year contracts
df.loc[df['contract_month'] == 12, 'remaining_time'] = 12 - (df['tenure'] % 12)

# Calculate the remaining time for two-year contracts
df.loc[df['contract_month'] == 24, 'remaining_time'] = 24 - (df['tenure'] % 24)

# Determine if the contract is near the end (remaining time <= 1 month)
df['near_contract_end'] = df['remaining_time'].apply(lambda x: 'yes' if x <= 1 else 'no')

# New Feature 3
# Convert 'yes'/'no' in StreamingMovies and StreamingTV to 1/0
df['streaming_movies_int'] = df['StreamingMovies'].apply(lambda x: 1 if x == 'yes' else 0)
df['streaming_tv_int'] = df['StreamingTV'].apply(lambda x: 1 if x == 'yes' else 0)

# Add StreamingMovies and StreamingTV usage
df['streaming_total'] = df['streaming_movies_int'] + df['streaming_tv_int']

# Create a binary indicator for Fiber optic (1 if Fiber optic, 0 otherwise)
df['fiber_optic'] = df['InternetService'].apply(lambda x: 1 if x == 'Fiber optic' else 0)

# Calculate the interaction term
df['internet_streaming_interaction'] = df['fiber_optic'] * df['streaming_total']

#df[df['internet_streaming_interaction']!='0']

#New feature 4
df['auto_bill_Yes'] = df['PaymentMethod'].apply(lambda x: 1 if 'automatic' in x.lower() else 0)
df['PaperlessBilling_Yes'] = df['PaperlessBilling'].apply(lambda x: 1 if x == 'yes' else 0)

df['billing_convenience'] = df.apply(lambda row: 2 if row['auto_bill_Yes'] == 1 and row['PaperlessBilling_Yes'] == 1
                                     else (1 if row['auto_bill_Yes'] == 1 or row['PaperlessBilling_Yes'] == 1 else 0), axis=1)

In [ ]:
df_ready = df_ready[~df_ready['TotalCharges'].isin([' ', '', '  '])]
df_ready[['MonthlyCharges', 'TotalCharges']] = df_ready[['MonthlyCharges', 'TotalCharges']].astype(float)
df_ready.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7032 entries, 0 to 7042
Data columns (total 18 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Contract                        7032 non-null   int64  
 1   InternetService                 7032 non-null   object 
 2   StreamingMovies                 7032 non-null   object 
 3   billing_convenience             7032 non-null   int64  
 4   OnlineSecurity                  7032 non-null   object 
 5   MonthlyCharges                  7032 non-null   float64
 6   TotalCharges                    7032 non-null   float64
 7   Dependents                      7032 non-null   object 
 8   PaymentMethod                   7032 non-null   object 
 9   renewed_status                  7032 non-null   int64  
 10  TechSupport                     7032 non-null   object 
 11  near_contract_end               7032 non-null   object 
 12  PaperlessBilling                7032 no

/var/folders/8t/3xrvdszj1g75llc2jb40dnvw0000gn/T/ipykernel_39665/2605809989.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ready[['MonthlyCharges', 'TotalCharges']] = df_ready[['MonthlyCharges', 'TotalCharges']].astype(float)
